In [1]:
# Install the necessary libraries
##!pip install sqlalchemy pandas surprise scikit-surprise textblob spacy python-telegram-bot nest-asyncio

# Download spaCy model
#!python -m spacy download en_core_web_sm

# Import the libraries
import threading
import asyncio
import pandas as pd
from surprise import Dataset, Reader, SVD
import urllib.request
import zipfile
import os
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from textblob import TextBlob
import spacy
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackContext
import nest_asyncio
from google.colab import userdata

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Apply the patch to allow the use of asyncio in Jupyter Notebook
nest_asyncio.apply()

# Configure the SQLite database
DATABASE_URL = 'sqlite:///movies.db'
engine = create_engine(DATABASE_URL)
Base = declarative_base()

# Define the ORM classes
class Movie(Base):
    __tablename__ = 'movies'
    movieId = Column(Integer, primary_key=True)
    title = Column(String)
    genres = Column(String)

class Rating(Base):
    __tablename__ = 'ratings'
    id = Column(Integer, primary_key=True, autoincrement=True)
    userId = Column(Integer, index=True)
    movieId = Column(Integer, index=True)
    rating = Column(Float)
    timestamp = Column(Integer)

# Create the tables in the database
Base.metadata.create_all(engine)

# Configure the session
Session = sessionmaker(bind=engine)
session = Session()

# Download and unzip the MovieLens dataset
url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
urllib.request.urlretrieve(url, 'ml-latest-small.zip')

with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall()

# Full path to the files
movies_file = os.path.join('ml-latest-small', 'movies.csv')
ratings_file = os.path.join('ml-latest-small', 'ratings.csv')

# Load the data
movies = pd.read_csv(movies_file)
ratings = pd.read_csv(ratings_file)

# Insert the data into the database
movies.to_sql('movies', engine, if_exists='replace', index=False)
ratings.to_sql('ratings', engine, if_exists='replace', index=False)

# Prepare the data for the Surprise library
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Split the data into training and test sets
trainset = data.build_full_trainset()

# Train the SVD model
algo = SVD()
algo.fit(trainset)

# Function to get movie recommendations from the database
def get_movie_recommendations_from_db(movie_id, num_recommendations=5):
    users_who_rated_movie = session.query(Rating.userId).filter(Rating.movieId == movie_id).distinct().all()
    users_who_rated_movie = [u[0] for u in users_who_rated_movie]

    other_movie_ids = session.query(Rating.movieId).filter(Rating.userId.in_(users_who_rated_movie)).distinct().all()
    other_movie_ids = [m[0] for m in other_movie_ids]

    predicted_ratings = []
    for mid in other_movie_ids:
        if mid != movie_id:
            predicted_ratings.append((mid, algo.predict(uid=0, iid=mid).est))

    predicted_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n_movies = [movie_id for movie_id, rating in predicted_ratings[:num_recommendations]]
    recommended_movies = session.query(Movie.title).filter(Movie.movieId.in_(top_n_movies)).all()
    return [m[0] for m in recommended_movies]

# Function to check if a movie title exists
def check_movie_title(title, session):
    result = session.query(Movie).filter(Movie.title.ilike(f"%{title}%")).all()
    return result

# Function for sentiment analysis
def analyze_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment

# Function for named entity recognition
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Token para o seu bot
TOKEN = userdata.get('YOUR_TELEGRAM_BOT_TOKEN')

# Function to start the bot
async def start(update: Update, context: CallbackContext) -> None:
    await update.message.reply_text(
        "Welcome to the Movie Recommendation CLI!\n"
        "Enter a movie title or a message (or 'exit' to quit):"
    )

# Function for handling text messages
async def handle_message(update: Update, context: CallbackContext) -> None:
    user_input = update.message.text.strip()
    if user_input.lower() == 'exit':
        await update.message.reply_text("Goodbye!")
        return

    # Sentiment analysis
    sentiment = analyze_sentiment(user_input)
    await update.message.reply_text(f"Sentiment Analysis: Polarity = {sentiment.polarity}, Subjectivity = {sentiment.subjectivity}")

    # Recognition of named entities
    entities = extract_entities(user_input)
    await update.message.reply_text(f"Named Entities: {entities}")

    # Movie search
    found_movies = check_movie_title(user_input, session)
    if found_movies:
        context.user_data['found_movies'] = found_movies
        movie_list = '\n'.join(f"{idx+1}. {movie.title} (ID: {movie.movieId})" for idx, movie in enumerate(found_movies))
        await update.message.reply_text(f"Found {len(found_movies)} movie(s):\n{movie_list}\n\nPlease enter the number of the movie for recommendations:")
    else:
        await update.message.reply_text(f"No movies found with title '{user_input}'. Please try again.")

# Function to recommend movies
async def recommend_movies(update: Update, context: CallbackContext) -> None:
    try:
        idx = int(update.message.text.strip()) - 1
        found_movies = context.user_data.get('found_movies', [])
        if 0 <= idx < len(found_movies):
            movie = found_movies[idx]
            recommendations = get_movie_recommendations_from_db(movie.movieId)
            if recommendations:
                rec_message = "\n".join(f"{i+1}. {rec}" for i, rec in enumerate(recommendations))
                await update.message.reply_text(f"Recommendations for '{movie.title}':\n{rec_message}")
            else:
                await update.message.reply_text(f"No recommendations found for '{movie.title}'.")
        else:
            await update.message.reply_text("Invalid selection. Please enter a valid number.")
    except (ValueError, IndexError):
        await update.message.reply_text("Invalid selection. Please enter a valid number.")

    await update.message.reply_text("Enter a movie title or a message (or 'exit' to quit):")

# Function to run the bot
def run_bot():
    asyncio.set_event_loop(asyncio.new_event_loop())
    application = Application.builder().token(TOKEN).build()
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.Regex(r'^\d+$'), recommend_movies))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
    asyncio.get_event_loop().run_until_complete(application.run_polling(stop_signals=None))

# Bot configuration
def main():
    thread = threading.Thread(target=run_bot)
    thread.start()

if __name__ == '__main__':
    main()

<ipython-input-1-8b48c32f09ca>:34: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
